In [1]:
import plotly.express as px
import pandas as pd

# Read Data

In [2]:
tree_loss_df = pd.read_csv('TreeCoverLoss_2001-2020_ByRegion.csv')

In [3]:
tree_loss_df.head()

,CountryCode,Year,TreeCoverLoss_ha,GrossEmissions_Co2_all_gases_Mg
0,AFG,2001,88.092712,2.226891e+04
1,AGO,2001,101220.621500,3.687662e+07
2,AIA,2001,3.878461,6.267003e+02
3,ALA,2001,396.934826,7.589787e+04
4,ALB,2001,3729.021031,1.333223e+06


In [4]:
country_code_info_df = pd.read_csv('country_code_info.csv')

In [5]:
country_code_info_df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [6]:
merged_df = tree_loss_df.merge(
    country_code_info_df[['alpha-3', 'region', 'sub-region']],
    left_on='CountryCode',
    right_on='alpha-3',
    how='inner'
)

In [7]:
merged_df.head()

,CountryCode,Year,TreeCoverLoss_ha,GrossEmissions_Co2_all_gases_Mg,alpha-3,region,sub-region
0,AFG,2001,88.092712,22268.90538,AFG,Asia,Southern Asia
1,AFG,2002,178.640364,50016.93763,AFG,Asia,Southern Asia
2,AFG,2003,244.336255,72753.97960,AFG,Asia,Southern Asia
3,AFG,2004,201.444959,55078.33929,AFG,Asia,Southern Asia
4,AFG,2005,235.956834,68037.96326,AFG,Asia,Southern Asia


# Visualize tree cover loss by country

In [8]:
fig = px.choropleth(
    merged_df,
    locations="CountryCode",                  
    locationmode="ISO-3",                    
    color="TreeCoverLoss_ha",               
    animation_frame="Year",                
    title="Global Tree Cover Loss by Country(2001-2020)", 
    color_continuous_scale="Reds",     
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)"}, 
    hover_name="region"               
)

fig.show()

In [9]:
top_10_countries_per_year = (
    merged_df.groupby("Year", group_keys=False)
    .apply(lambda x: x.nlargest(10, "TreeCoverLoss_ha"))
    .reset_index()
)

fig = px.bar(
    top_10_countries_per_year,
    x="TreeCoverLoss_ha",           
    y="CountryCode",           
    color="region",             
    animation_frame="Year",       
    title="Top 10 Countries by Tree Cover Loss (2001-2020)",
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)", "CountryCode": "Country Code"},
    orientation="h",           
)

fig.update_layout(
    xaxis_title="Tree Cover Loss (ha)", 
    yaxis_title="Country Code", 
    yaxis=dict(categoryorder="total ascending"),  
    showlegend=True           
)

fig.show()

# Visualize by State

In [10]:
region_year_loss = merged_df.groupby(['Year', 'region'])['TreeCoverLoss_ha'].sum().reset_index()

merged_df_with_year_loss = merged_df.merge(
    region_year_loss,
    on=['Year', 'region'],
    suffixes=('', '_total')
)

final_map_df = merged_df_with_year_loss.drop_duplicates(subset=["CountryCode", "Year"])

fig = px.choropleth(
    final_map_df,
    locations="alpha-3",              
    locationmode="ISO-3",                
    color="TreeCoverLoss_ha_total",    
    hover_name="region",          
    animation_frame="Year",      
    title="Global Tree Cover Loss by Region (2001-2020)", 
    color_continuous_scale="Reds",   
    labels={"TreeCoverLoss_ha_total": "Total Tree Cover Loss (ha)"}
)

fig.show()

In [11]:
fig = px.bar(
    region_year_loss,
    x="TreeCoverLoss_ha",       
    y="region",                
    color="region",            
    animation_frame="Year",      
    title="Global Tree Cover Loss by Region (2001-2020)",  
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)", "region": "Region"},
    orientation="h"           
)

fig.update_layout(
    xaxis_title="Tree Cover Loss (ha)",  
    yaxis_title="Region",         
    yaxis=dict(categoryorder="total ascending"),  
    showlegend=False              
)

fig.show()


# Visualize by sub-region

In [12]:
subregion_year_loss = merged_df.groupby(['Year', 'sub-region'])['TreeCoverLoss_ha'].sum().reset_index()

merged_df_with_subregion_loss = merged_df.merge(
    subregion_year_loss,
    on=['Year', 'sub-region'],
    suffixes=('', '_total') 
)

final_map_df = merged_df_with_subregion_loss.drop_duplicates(subset=["CountryCode", "Year"])

fig = px.choropleth(
    final_map_df,
    locations="alpha-3",                  
    locationmode="ISO-3",              
    color="TreeCoverLoss_ha_total",       
    hover_name="sub-region",             
    animation_frame="Year",             
    title="Global Tree Cover Loss by Sub-Region (2001-2020)", 
    color_continuous_scale="Reds",     
    labels={"TreeCoverLoss_ha_total": "Total Tree Cover Loss (ha)"}
)

fig.show()


In [13]:
fig = px.bar(
    subregion_year_loss,
    x="TreeCoverLoss_ha",        
    y="sub-region",             
    color="sub-region",         
    animation_frame="Year",    
    title="Global Tree Cover Loss by Sub-Region (2001-2020)", 
    labels={"TreeCoverLoss_ha": "Tree Cover Loss (ha)", "sub-region": "Sub-Region"},
    orientation="h"         
)

fig.update_layout(
    xaxis_title="Tree Cover Loss (ha)", 
    yaxis_title="Sub-Region",  
    yaxis=dict(categoryorder="total ascending"), 
    showlegend=False   
)

fig.show()